# lucj_energy.ipynb
In this notebook, we will demonstrate how to use `QuCES` to compute the energy of a Local Unitary Cluster Jastrow circuit simulation.

In [6]:
import pyscf
import ffsim
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, QuantumRegister

import quces.utils as qcu

# Define molecule with pyscf
atom = lambda R: [["H", (0, 0, 0)], ["H", (0, 0, R)]]
R = 0.75  
mol = pyscf.gto.Mole()
mol.build(atom=atom(R),basis="sto-3g",unit="Angstrom")
active_space = range(0, mol.nao_nr()) # complete active space

# Run scf in pyscf and obtain molecular data with ffsim
scf = pyscf.scf.RHF(mol).run()
mol_data = ffsim.MolecularData.from_scf(scf, active_space=active_space)
norb     = mol_data.norb
nelec = mol_data.nelec

# Define Qubit Hamiltonian with ffsim molecular data 
H_f = ffsim.fermion_operator(mol_data.hamiltonian) # second quantized fermionic Molecular hamiltonian
H = ffsim.qiskit.jordan_wigner(H_f, norb=norb)

# Define the LuCJ ansatz with ffsim
pairs_aa = [(p, p + 1) for p in range(norb - 1)]
pairs_ab = [(p, p) for p in range(norb)]
interaction_pairs = (pairs_aa, pairs_ab)
frozen_list = [i for i in range(mol.nao_nr()) if i not in active_space]
ccsd = pyscf.cc.CCSD(scf, frozen=frozen_list).run()
ucj = ffsim.UCJOpSpinBalanced.from_t_amplitudes(t2=ccsd.t2, t1=ccsd.t1, n_reps=1, interaction_pairs=interaction_pairs)
n_qubits = 2 * norb
qreg = QuantumRegister(n_qubits, "q")

lucj_circuit = QuantumCircuit(qreg)
lucj_circuit.append(ffsim.qiskit.PrepareHartreeFockJW(norb, nelec), qreg)
lucj_circuit.append(ffsim.qiskit.UCJOpSpinBalancedJW(ucj), qreg)

# Define Qiskit-Aer simulator
simulator = AerSimulator(method="matrix_product_state")

converged SCF energy = -1.1161514489386
E(CCSD) = -1.137117245163172  E_corr = -0.02096579622456951


In [8]:
# QuCES expectation value of circuit energy
lucj_energy = qcu.circuit_energy(simulator, lucj_circuit, H)

print(f"---- MPS LuCJ circuit simulation energy ----")
print(f"Hydrogen at R = 0.75 Å: E = {lucj_energy} Ha")

---- MPS LuCJ circuit simulation energy ----
Hydrogen at R = 0.75 Å: E = -1.1253359545407071 Ha
